In [23]:
import numpy as np
from iminuit import Minuit
from classy import Class
import act_dr6_spt_lenslike as apslike

n_om = 100
om_min = .05
om_max = 1

n_s8 = 100
s8_min = 0.4
s8_max = 1.2

n_gamma = 100
gamma_min = -0.5
gamma_max = 2

omega_vals = np.asarray(np.linspace(om_min, om_max, n_om))
sigma_vals = np.asarray(np.linspace(s8_min, s8_max, n_s8))
gamma_vals = np.asarray(np.linspace(gamma_min, gamma_max, n_gamma))

In [80]:
params = {
    'h': 0.674,
    'omega_b': 0.022,
    'omega_cdm': 0.12,
    'A_s': 2.1e-9,
    'n_s': 0.965,
    'tau_reio': 0.054,
    'output': 'tCl pCl lCl',
    'lensing': 'yes'
}

cosmo = Class()
cosmo.set(params)
cosmo.compute()

lmax = 2999
cls = cosmo.raw_cl(lmax)
cl_pp = cls['pp'] * 1e12   # µK²
ell = cls['ell']

cl_kk_theory = 0.25 * (ell * (ell + 1))**2 * cl_pp

cosmo.struct_cleanup()
cosmo.empty()

In [ ]:
def chi2_lensing(cl_kk_theory, data_binned_clkk, binmat_act, cinv):
    model_binned = binmat_act @ cl_kk_theory
    residual = data_binned_clkk - model_binned
    return residual.T @ cinv @ residual

variant = 'act_baseline'
lens_only = False # use True if not combining with any primary CMB data
like_corrections = True # should be False if lens_only is True

data_dict = apslike.load_data(variant, lens_only=lens_only, like_corrections=like_corrections)

data = data_dict['data_binned_clkk']
binning_mat = data_dict['binmat_act']
cov_mat_inv = data_dict['cinv']

Loading ACT DR6 lensing likelihood v1.2...


In [82]:
# data_dict.keys()
cl_kk_theory.shape

for x in [data, binning_mat, cov_mat, cov_mat_inv, cl_kk_theory]:
    print(x.shape)

print(chi2_lensing(cl_kk_theory, data, binning_mat, cov_mat_inv))

(10,)
(10, 3000)
(10, 10)
(10, 10)
(3000,)
1.8131678967820463e+27


In [10]:
import act_dr6_spt_lenslike as apslike

variant = 'act_baseline'
lens_only = False # use True if not combining with any primary CMB data
like_corrections = True # should be False if lens_only is True

# Do this once
data_dict = apslike.load_data(variant,lens_only=lens_only,like_corrections=like_corrections)
# This dict will now have entries like `data_binned_clkk` (binned data vector), `cov`
# (covariance matrix) and `binmat_act` (binning matrix to be applied to a theory
# curve starting at ell=0).

# Get cl_kk, cl_tt, cl_ee, cl_te, cl_bb predictions from your Boltzmann code.
# These are the CMB lensing convergence spectra (not potential or deflection)
# as well as the TT, EE, TE, BB CMB spectra (needed for likelihood corrections)
# in uK^2 units. All of these are C_ell (not D_ell), no ell or 2pi factors.
# Then call
# lnlike=apslike.generic_lnlike(data_dict,ell_kk,cl_kk,ell_cmb,cl_tt,cl_ee,cl_te,cl_bb)

Loading ACT DR6 lensing likelihood v1.2...


/home/vicpabo/Documents/Projects/Stage-irap/programs/virtual_env/lib/python3.13/site-packages/act_dr6_spt_lenslike/act_dr6_spt_lenslike.py:500: UserWarning: Hartlap correction to cinv: 0.9861635220125786
  warnings.warn(f"Hartlap correction to cinv: {hartlap_correction}")
